# Homeworks

Base code for basic data we used during the whole week

In [3]:
#Import statements - To keed this in order
from sklearn.metrics import mean_absolute_error
from nltk.corpus import opinion_lexicon
import urllib.request, os, gzip
import  json
import random
import numpy # a powerfull module
import  tensorflow  as tf # Googles ML module
from nltk.corpus import stopwords # We use it to remove the stopwords of the comments since they dont provide relevant info
from nltk.tokenize import sent_tokenize, word_tokenize , TweetTokenizer # Divide text in sentences and then in words
from sklearn.linear_model import LinearRegression # sklearn is a machine learning toolkit (needs numpy, scipy and matplotlib)
from sklearn.linear_model import LogisticRegression # http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
# If I am not mistaken, logistic regresion is for booleans clasiffiers
from  sklearn.metrics import precision_score, recall_score # I think these are metrics to evaluate models form the slklearn library
from sklearn.naive_bayes import GaussianNB, BernoulliNB # To apply statistical models, not regressions(?)1
from  nltk.sentiment  import SentimentAnalyzer
from nltk.sentiment.util import mark_negation, HAPPY, SAD #Utilities of negation and happy or sad emojis
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [4]:
%matplotlib inline
datadir = './data/'

def download_data(dataset_name, datadir):
    filename = 'reviews_%s_5.json' % dataset_name
    filepath = os.path.join(datadir, filename)
    if os.path.exists(filepath):
        print("Dataset %s has already been downloaded to %s" % (dataset_name, datadir))
    else:
        url = 'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/%s.gz' % filename
        urllib.request.urlretrieve(url, filepath + ".gz")
        with gzip.open(filepath + ".gz", 'rb') as fin:
            with open(filepath, 'wb') as fout:
                fout.write(fin.read())
        print("Downloaded dataset %s and saved it to %s" % (dataset_name, datadir))

dataset = "Baby"
download_data(dataset, datadir)

Dataset Baby has already been downloaded to ./data/


In [5]:
def  load_data (dataset_name, datadir):
    filepath = os.path.join(datadir, 'reviews_%s_5.json' % dataset_name)
    if not os.path.exists(filepath):
        download_data(dataset_name, datadir)
    data = []
    with open(filepath, 'r') as f:
        for line in f:                            # read file line by line
            item_hash = hash(line)                # we will use this later for partitioning our data 
            item = json.loads(line)               # convert JSON string to Python dict
            item['hash'] = item_hash              # add hash for identification purposes
            data.append(item)
    print("Loaded %d data for dataset %s" % (len(data), dataset_name))
    return data

# load the data...
baby = load_data(dataset, datadir)

Loaded 160792 data for dataset Baby


In [6]:
def partition_train_validation_test(data):
    # 60% : modulus is 0, 1, 2, 3, 4, or 5
    data_train = [item for item in data if item['hash']%10<=5]  
    # 20% : modulus is 6 or 7
    data_valid = [item for item in data if item['hash']%10 in [6,7]] 
    # 20% : modulus is 8 or 9
    data_test  = [item for item in data if item['hash']%10 in [8,9]] 
    print("Now we have", len(data_train), "training examples,", len(data_valid),
      "validation examples, and", len(data_test), "test examples")
    return data_train, data_valid, data_test
    
baby_train, baby_valid, baby_test = partition_train_validation_test(baby)




Now we have 96682 training examples, 31721 validation examples, and 32389 test examples


In [7]:
eng_stopwords = set(stopwords.words('english'))
positive_words = set(opinion_lexicon.positive())
negative_words = set(opinion_lexicon.negative())

def my_tokenize(text):
    # split text into lower-case tokens, removing all-punctuation tokens and stopwords
    tokens = []
    for sentence in sent_tokenize(text):
        #Adds to the array and array with the words in lowercase, we add them if they are not stopwords and there is at least one letter in it
        tokens.extend(x for x in word_tokenize(sentence.lower()) #continues down...
                      if x not in eng_stopwords and any(i.isalpha() for i in x))# This extends the list by adding elements, it is different from append... see https://stackoverflow.com/questions/252703/difference-between-append-vs-extend-list-methods-in-python 
    return tokens

def pos_neg_fraction(text): # We recieve the raw text
    tokens = my_tokenize(text) # We tokenize the text first
    count_pos, count_neg = 0, 0
    for t in tokens:
        if t in positive_words:
            count_pos += 1
        if t in negative_words:
            count_neg += 1
    count_all = len(tokens) # this is because we need to be sure there is no 0 len sentence
    if count_all != 0:
        return count_pos/count_all, count_neg/count_all
    else:
        return 0., 0.
    
pos_example = 'This is a good, great, fantastic, amazing, wonderful, super product!!!'
neg_example = 'This is a bad, atrocious, terrible, dreadful, awful, abysmal product!!!'
print(pos_neg_fraction(pos_example))
print(pos_neg_fraction(neg_example))

(0.8571428571428571, 0.0)
(0.0, 0.8571428571428571)


In [8]:
def dataset_to_matrix(data):
    # data is a lot of text in {} that has reviwer name..comment...date...etc.. but all identified with text labels
    #in that sence data is an unidiminsional array
    # the item "atribute", we added it to the data to identify the sections of it
    return numpy.array([list(pos_neg_fraction(item['reviewText'])) for item in data])
# X_train with two columns and as many rows as there are examples in the data set. 
#The first column contains the fraction of positive words, 
#while the second column contains the fraction of negative words for each example.
X_train = dataset_to_matrix(baby_train)
X_valid = dataset_to_matrix(baby_valid)
X_test = dataset_to_matrix(baby_test)
most_pos, most_neg = numpy.argmax(X_train, axis=0) # find maximum ROW (axis 0).. through aaaaallll the data (OMG!)
# print the example with the highest fraction of positive words:
print("We found a fraction of %f %% positive words for example %d" % 
      (100.*X_train[most_pos, 0], most_pos))
print(baby_train[most_pos])
print("We found a fraction of %f %% negative words for example %d" %
      (100.*X_train[most_neg, 1], most_neg))
print(baby_train[most_neg])

We found a fraction of 100.000000 % positive words for example 22847
{'reviewerID': 'A3I9BADI3XRAAV', 'asin': 'B000S9RF4M', 'reviewerName': 'Lucky', 'helpful': [0, 0], 'reviewText': 'Love if', 'overall': 5.0, 'summary': 'Five Stars', 'unixReviewTime': 1404864000, 'reviewTime': '07 9, 2014', 'hash': 2889226928932461534}
We found a fraction of 100.000000 % negative words for example 24892
{'reviewerID': 'A1SLEYD29KEUW1', 'asin': 'B000WUD83O', 'reviewerName': 'ABDULLAH AL-FALAH', 'helpful': [0, 0], 'reviewText': 'too noisy', 'overall': 2.0, 'summary': 'Two Stars', 'unixReviewTime': 1404432000, 'reviewTime': '07 4, 2014', 'hash': 6995570254324240972}


In [9]:
def  dataset_to_targets (data):
    return numpy.array([item['overall'] for item in data])

Y_train = dataset_to_targets(baby_train)
Y_valid = dataset_to_targets(baby_valid)
Y_test = dataset_to_targets(baby_test)
print("Our feature matrix is two-dimensional and has shape", X_train.shape) # contains pos,neg fraction
print("Our target vector is one-dimensional and has shape", Y_train.shape) # containd sscore

Our feature matrix is two-dimensional and has shape (96682, 2)
Our target vector is one-dimensional and has shape (96682,)


## Day 1

In [10]:
lreg = LinearRegression().fit(X_train,Y_train)
print("The coefficient for the fpos variable is", lreg.coef_[0])
print("The coefficient for the fneg variable is", lreg.coef_[1])
print("The intercept is", lreg.intercept_)

The coefficient for the fpos variable is 3.1873335424228184
The coefficient for the fneg variable is -5.711666718416578
The intercept is 4.0212270116731


In [11]:
#If the review contains 20% positive words (fpos==0.2) 
#but still no negative words (fneg==0), we would expect the following rating:
features = [[0.2, 0]]
expected_rating_A = lreg.predict(features)[0]
print("The expected rating is %f stars" % expected_rating_A)
# we can also compute this explicitly:
expected_rating_B = lreg.intercept_ + 0.2*lreg.coef_[0] + 0*lreg.coef_[1]
print("This is the same as %f stars" % expected_rating_B)
#However, if the review contains no positive words (fpos==0) but 20% negative words (fneg==0.2),
#we expect the following rating:
features = [[0, 0.2]]
expected_rating_A = lreg.predict(features)[0]
print("The expected rating is %f stars" % expected_rating_A)
# we can also compute this explicitly:
expected_rating_B = lreg.intercept_ + 0 * lreg.coef_[0] + 0.2 * lreg.coef_[1]
print("This is the same as %f stars" % expected_rating_B)

The expected rating is 4.658694 stars
This is the same as 4.658694 stars
The expected rating is 2.878894 stars
This is the same as 2.878894 stars


In [12]:
def predict_lreg(features):
    expected_rating = lreg.predict(features)
    expected_rating[expected_rating > 5.0] = 5.0
    expected_rating[expected_rating < 1.0] = 1.0
    return expected_rating

pred_train = predict_lreg(X_train)

In [13]:
mae_train = mean_absolute_error(pred_train, Y_train)
print("The mean absolute error on the training data is %f stars" % mae_train)

The mean absolute error on the training data is 0.827829 stars


#### - calculate the prediction for 100% pos, and 100% neg review

In [14]:
features_100pos=[[1,0]]
features_100neg=[[0,1]]
expected_rating_pos = lreg.predict(features_100pos)[0] # I think 0 is for linear as in kernel = {‘linear’, ‘rbf’, ‘poly’, ‘sigmoid’, ‘precomputed’}
expected_rating_neg = lreg.predict(features_100neg)[0]
print("The expected rating for 100 pos review is %f stars" % expected_rating_pos)
print("The expected rating  for 100 neg review is %f stars" % expected_rating_neg)
# This model needs a threshold to limit the starts between 0 and 5

The expected rating for 100 pos review is 7.208561 stars
The expected rating  for 100 neg review is -1.690440 stars


#### - Repeat this same process for "Apps for Android" dataset

In [15]:
###### Getting data####
android_dataset = "Apps_for_Android"
download_data(android_dataset, datadir)

Dataset Apps_for_Android has already been downloaded to ./data/


In [16]:
################Processing data############
# Load data from directory
apps= load_data(android_dataset,datadir)
#Partition data for training validation and tests
apps_train,apps_valid,apps_test = partition_train_validation_test(apps)
#We get the portions of positive and negative terms, omiting stopwords and puntuation marks (tokenizing words and sentences as well))
X_apps_train = dataset_to_matrix(apps_train)
app_most_pos, app_most_neg = numpy.argmax(X_apps_train, axis=0)
print(apps_train[app_most_pos])
print()
print(apps_train[app_most_neg])
Y_apps_train = dataset_to_targets(apps_train) # extract the overall item from each review

Loaded 752937 data for dataset Apps_for_Android
Now we have 451818 training examples, 150890 validation examples, and 150229 test examples
{'reviewerID': 'AWUDE6LOH5Y2Q', 'asin': 'B004DLPXAO', 'reviewerName': 'Amazon Customer', 'helpful': [0, 0], 'reviewText': 'Love it!', 'overall': 5.0, 'summary': 'Take you Kindle Library along whenever you have your phone!', 'unixReviewTime': 1404691200, 'reviewTime': '07 7, 2014', 'hash': 2546480984808772441}

{'reviewerID': 'A1HZYCWWJ53TS5', 'asin': 'B004HE5TAG', 'helpful': [0, 0], 'reviewText': 'boring', 'overall': 2.0, 'summary': 'Two Stars', 'unixReviewTime': 1405382400, 'reviewTime': '07 15, 2014', 'hash': 7797960231240202205}


In [17]:
####Linear regression
lreg_app = LinearRegression().fit(X_apps_train, Y_apps_train)
app_pos_features = [[0.2, 0]]
app_neg_features = [[0,0.2]]
expected_rating_pos = lreg.predict(app_pos_features)[0]
expected_rating_neg= lreg.predict(app_neg_features)[0]
print("The expected rating for a review with 20 porcet positive comments is %f stars" % expected_rating_pos)
print("The expected rating for a review with 20 porcet negative comments is %f stars" % expected_rating_neg)
#With this data we get sort of a similiar approximation

The expected rating for a review with 20 porcet positive comments is 4.658694 stars
The expected rating for a review with 20 porcet negative comments is 2.878894 stars


In [18]:
pred_train_apps = predict_lreg(X_apps_train)
mae_train_apps = mean_absolute_error(pred_train_apps, Y_apps_train)
print("The mean absolute error on the training data is %f stars" % mae_train_apps)
#We are getting even a higher error with a much more bigger dataset, funny, because it is bigger

The mean absolute error on the training data is 0.941976 stars


# Day 2

In [19]:
def discretize_targets(Y):
    return Y<= 3.0 # So this is a boolean, we are returning if the conditions is true
D_train = discretize_targets(Y_train) # We must remember that Y_train is a 1 dimension array with the scores
print("The training data contains %f %% dissatisfied customers" % (100.*D_train.mean()))
print(D_train[:10])

The training data contains 21.219048 % dissatisfied customers
[False False False False False False  True  True False False]


In [20]:
logreg = LogisticRegression().fit(X_train, D_train) # We use just the training data
# The predict_proba() method produces a matrix with two columns
# the first column contains the probability for the label being "false" (satisfied customer)
# the second column contains the probability for the label being "true" (dissatisfied customer)
# the sum of both columns is 1
# we select the second column with [:,1]
# [:,0] would select the first column
# [1,:] would select the second row
prob2_train = logreg.predict_proba(X_train)[:,1]
pred2_train = prob2_train > 0.5
max_prob2 = numpy.argmax(prob2_train)
min_prob2 = numpy.argmin(prob2_train)
def analyze_training_example_2(i):
    print("Training example number", i)
    print("True rating = %f stars" % Y_train[i])
    print("Expected to be dissatisfied:", pred2_train[i])
    print("Expected probability of being dissatisfied : %f" % prob2_train[i])
    print("Features = %f / %f" % (X_train[i,0], X_train[i,1]))
    print("Review text = %s" % baby_train[i]['reviewText'])
    
analyze_training_example_2(max_prob2)
print()
analyze_training_example_2(min_prob2)

Training example number 24892
True rating = 2.000000 stars
Expected to be dissatisfied: True
Expected probability of being dissatisfied : 0.999863
Features = 0.000000 / 1.000000
Review text = too noisy

Training example number 22847
True rating = 5.000000 stars
Expected to be dissatisfied: False
Expected probability of being dissatisfied : 0.000038
Features = 1.000000 / 0.000000
Review text = Love if


In [21]:
precision2 = precision_score(D_train, pred2_train)
recall2 = recall_score(D_train, pred2_train)
print("For the default threshold (0.5) we get precision = %f "
      "and recall = %f" % (precision2, recall2))

For the default threshold (0.5) we get precision = 0.617335 and recall = 0.093736


In [22]:
nb = GaussianNB().fit(X_train, D_train)
prob3_train = nb.predict_proba(X_train)[:,1]
pred3_train = prob3_train>0.5
precision3 = precision_score(D_train, pred3_train)
recall3 = recall_score(D_train, pred3_train)
print("Now the precision is %f and the recall is %f" % (precision3, recall3))

Now the precision is 0.489264 and the recall is 0.155496


#### -Change the treshold from 0.5 to 0.2, and rerun the code.

In [23]:
new_pred2_train = prob2_train > 0.2
new_precision = precision_score(D_train, new_pred2_train)
new_recall =  recall_score(D_train, new_pred2_train)
print("For the default threshold (0.2) we get precision = %f "
      "and recall = %f" % (new_precision, new_recall))

For the default threshold (0.2) we get precision = 0.321286 and recall = 0.733610


#### - Give a commentary in plain English about how that changed precision and recall. What does that mean? What is now included that wasn't before? What part of it is good? What is bad from our Task perspective. Remember: our task was to identify Dissatisfied reviews.

The precision is droping because we are taking more cases as valid (dissatisfied costumers) as our threshold is lower, however there are more chances that a higher proportion of cases labaled as "dissatisfied" where actually sattisfied. From our problem perspective I think it is better to encrease the recall as I guess we are more interested in identifiying dissatisfied costumers to address the product's problems .

In [24]:
feat_cols = [tf.feature_column.numeric_column(key="fpos"),
             tf.feature_column.numeric_column(key="fneg")]
model = tf.estimator.LinearRegressor(feature_columns=feat_cols)
get_training_data = tf.estimator.inputs.numpy_input_fn(
                     x={"fpos" : X_train[:,0], "fneg" : X_train[:,1]},
                     y=Y_train, num_epochs=None, shuffle=True)

model.train(input_fn=get_training_data, steps=5000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Mara\\AppData\\Local\\Temp\\tmpmta65bsf', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000296007B2240>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into

In [25]:

eval_training_data = tf.estimator.inputs.numpy_input_fn(
                     x={"fpos":X_train[:,0], "fneg": X_train[:,1]},
                     num_epochs=1, shuffle=False)
pred_train_tf = numpy.array([item['predictions'][0] for item in 
                         model.predict(input_fn=eval_training_data)])
mae_train_tf = mean_absolute_error(pred_train_tf, Y_train)
print("The mean absolute error on the training data is %f stars" 
      % mae_train_tf)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Mara\AppData\Local\Temp\tmpmta65bsf\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
The mean absolute error on the training data is 0.839051 stars


In [26]:
examples_negation = ["This product wasn't bad.",
                     "This is not a bad product.",
                     "This product was bad.",
                     "This is a bad product."]

for sentence in examples_negation:
    tokens_with_negation = mark_negation(word_tokenize(sentence.lower())) # Append _NEG suffix to words that appear in the scope between a negation and a punctuation mark.
    print("Sentence =", sentence)
    print(tokens_with_negation)

negated_stopwords = set(x+"_NEG" for x in eng_stopwords)
all_stopwords = eng_stopwords.union(negated_stopwords)   # set union stopwords and negate stopwords
    
def tokenize_with_negation(text):
    # split text into lower-case tokens, removing all-punctuation tokens and stopwords
    tokens = []
    for sentence in sent_tokenize(text):
        pretokens = word_tokenize(sentence.lower())
        pretokens = [x for x in pretokens if any(i.isalpha() for i in x)] #Everything the same unitl here
        #Append _NEG suffix to words that appear in the scope between a negation 
        #and a punctuation mark
        pretokens = mark_negation(pretokens) #Here
        # ok, but tokens started empy so we are bsically storing the array
        tokens.extend(x for x in pretokens if x not in all_stopwords) # puts a list at the beegining of the frist list
    return tokens

print(baby_train[31]['reviewText'])
print(tokenize_with_negation(baby_train[31]['reviewText']))

Sentence = This product wasn't bad.
['this', 'product', 'was', "n't", 'bad_NEG', '.']
Sentence = This is not a bad product.
['this', 'is', 'not', 'a_NEG', 'bad_NEG', 'product_NEG', '.']
Sentence = This product was bad.
['this', 'product', 'was', 'bad', '.']
Sentence = This is a bad product.
['this', 'is', 'a', 'bad', 'product', '.']
I bought this last month for my 6 month old son, and it continues to fascinate him.  It is fabulous to encourage moving and crawling as I set them around him and they slide away as he reaches for them -- its been invaluable in encouraging him towards crawling and scooting. When he does get hold of them, exploring the ladder on the fire truck develops his fine motor skills too, and he like chewing on all of them and shaking the rattly little one!  He'll probably soon be able to work the sun roof. It will be a stacking toy later too! Great value for money...
['bought', 'last', 'month', 'month', 'old', 'son', 'continues', 'fascinate', 'fabulous', 'encourage', 

In [27]:
all_positive_words = positive_words.union({x+"_NEG" for x in negative_words})
all_negative_words = negative_words.union({x+"_NEG" for x in positive_words})
def pos_neg_fraction_with_negation(text):
    tokens = tokenize_with_negation(text) #Tokenize and adds negation when necessary
    # count how many positive and negative words occur in the text
    count_pos, count_neg = 0, 0
    for t in tokens:
        if t in all_positive_words:
            count_pos += 1
        if t in all_negative_words:
            count_neg += 1
    count_all = len(tokens)
    if count_all != 0:
        return count_pos/count_all, count_neg/count_all
    else:  # avoid division by zero
        return 0., 0.
    
pos_example = 'This is a good, great, fantastic, amazing, wonderful, super product!!!'
neg_example = 'This is a bad, atrocious, terrible, dreadful, awful, abysmal product!!!'
print(pos_neg_fraction_with_negation(pos_example))
print(pos_neg_fraction_with_negation(neg_example))
pos_example_neg = 'This is not a good, great, fantastic, amazing, wonderful, super product!!!'
neg_example_neg = 'This is not a bad, atrocious, terrible, dreadful, awful, abysmal product!!!'
print(pos_neg_fraction_with_negation(pos_example_neg))
print(pos_neg_fraction_with_negation(neg_example_neg))

(0.8571428571428571, 0.0)
(0.0, 0.8571428571428571)
(0.0, 0.8571428571428571)
(0.8571428571428571, 0.0)


In [28]:
def dataset_to_matrix_with_neg(data):
    return numpy.array([list(pos_neg_fraction_with_negation(item['reviewText'])) for item in data])

X_train_neg = dataset_to_matrix_with_neg(baby_train)

In [29]:
lreg_neg = LinearRegression().fit(X_train_neg, Y_train)
pred_train_neg = lreg_neg.predict(X_train_neg)
mae_train_with_neg = mean_absolute_error(pred_train_neg, Y_train)
print("Now the mean absolute error on the training data is %f stars" % mae_train_with_neg)

Now the mean absolute error on the training data is 0.812200 stars


In [30]:
# Random Forest
from sklearn.ensemble import RandomForestRegressor
rf_neg = RandomForestRegressor().fit(X_train_neg,Y_train)
pred_train_rf_neg = rf_neg.predict(X_train_neg)
mae_train_rf_neg = mean_absolute_error(pred_train_rf_neg, Y_train)
print("A nonlinear regressor achieves a MAE of %f stars" % mae_train_rf_neg)

A nonlinear regressor achieves a MAE of 0.712649 stars


#### Add features , Explain which features you chose, implement them, and write a commentary on your results... Feel welcome to use NLTK's built-in sentiment analyzer or any other research that you can find and understand

I will add emojis (personal choice, I communicate with emojis the whole time), also if a word is in uupercases then it will count as 2 times

In [31]:
def tokenize_with_negation_Emojis(text):
    # split text into lower-case tokens, removing all-punctuation tokens and stopwords
    tokens = []
    tknzr = TweetTokenizer()
    for sentence in sent_tokenize(text):
        #pretokens = word_tokenize(sentence.lower())
        emopretokens = tknzr.tokenize(sentence) #use tweet tokenizer to include emojis
        emojis  = [x for x in emopretokens if x in SAD or x in HAPPY]
        # if a word is in uppercase then count ir twice 
        pretokens_emphasis = [x for x in emopretokens if x.isupper() and x.isalpha()]
        pretokens = word_tokenize(sentence.lower())
        pretokens = [x for x in pretokens if any(i.isalpha() for i in x)] #Everything the same unitl here
        #Append _NEG suffix to words that appear in the scope between a negation 
        #and a punctuation mark
        pretokens.extend(pretokens_emphasis)
        pretokens = mark_negation(pretokens) #Here
        pretokens.extend(emojis)
        # ok, but tokens started empy so we are bsically storing the array
        tokens.extend(x for x in pretokens if x not in all_stopwords) # puts a list at the beegining of the frist list
    return tokens

all_positive_words_emoji = all_positive_words.union(HAPPY)
all_negative_words_emoji = all_negative_words.union(SAD)

def pos_neg_fraction_with_negation_emoji(text):
    tokens = tokenize_with_negation_Emojis(text) #Tokenize and adds negation when necessary
    # count how many positive and negative words occur in the text
    count_pos, count_neg = 0, 0
    for t in tokens:
        if t in all_positive_words_emoji:
            count_pos += 1
        if t in all_negative_words_emoji:
            count_neg += 1
    count_all = len(tokens)
    if count_all != 0:
        return count_pos/count_all, count_neg/count_all
    else:  # avoid division by zero
        return 0., 0.
def dataset_to_matrix_with_neg_emoji(data):
    return numpy.array([list(pos_neg_fraction_with_negation_emoji(item['reviewText'])) for item in data])


In [32]:
my_example =" :D :) ;) "
print(pos_neg_fraction_with_negation_emoji(my_example))
my_example = ":( :/)"
print(pos_neg_fraction_with_negation_emoji(my_example))
print(baby_train[700]['reviewText'])
print(pos_neg_fraction_with_negation_emoji(baby_train[700]['reviewText']))

(1.0, 0.0)
(0.0, 1.0)
I decided to use these because I wanted the Avent Breast Pump and would use these with it.  Well, my newborn had a hard time latching onto the nipples and when he did they were way too fast with the flow.  He would choke and gasp almost everytime.  I now use the Playtex Ventaire bottles and he doesn't have as much of a problem.  Every baby is different, though, but if you have a baby that tends to want to eat quickly, these won't work so well.
(0.06521739130434782, 0.10869565217391304)


In [33]:
X_train_neg_emoji = dataset_to_matrix_with_neg_emoji(baby_train)

In [34]:
lreg_neg_emoji = LinearRegression().fit(X_train_neg_emoji, Y_train)
pred_train_neg_emoji = lreg_neg_emoji.predict(X_train_neg_emoji)
mae_train_with_neg_emoji = mean_absolute_error(pred_train_neg_emoji, Y_train)
print("Now the mean absolute error on the training data is %f stars" % mae_train_with_neg_emoji)
# I managed to improve the model a littttllleee bit :P 0.813530 

Now the mean absolute error on the training data is 0.812148 stars


In [35]:
#Diferences between the tokenizers
print(TweetTokenizer().tokenize("this is HUGE!! :D "))
print(word_tokenize("this is HUGE!! :D"))

['this', 'is', 'HUGE', '!', '!', ':D']
['this', 'is', 'HUGE', '!', '!', ':', 'D']


# DAy3

In [36]:
sia= SentimentIntensityAnalyzer()
text = baby_train[50000]['reviewText']
for s in sent_tokenize(text):
    print(s)
    print(sia.polarity_scores(s))
    

Survived 2 flights and a lot of walking/strolling in Las Vegas so I feel like I have already gotten my money's worth.
{'neg': 0.0, 'neu': 0.653, 'pos': 0.347, 'compound': 0.7893}
Easy to push, easy to fold/unfold.
{'neg': 0.0, 'neu': 0.408, 'pos': 0.592, 'compound': 0.7003}
Fairly lightweight.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Cargo basket is standard for umbrella type strollers.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Parent pouch came in handy.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Really happy with this purchase.
{'neg': 0.0, 'neu': 0.5, 'pos': 0.5, 'compound': 0.6115}


In [37]:
def sia_features(dataset):
    '''
    For each review text in the dataset, extract: 
    (1) mean positive sentiment over all sentences
    (2) mean neutral sentiment over all sentences
    (3) mean negative sentiment over all  sentences
    (4) maximum postive sentiment over all sentences
    (5) maximum neutral sentiment over all sentences
    (6) maximum negative sentiment over al sentences 
'''
    feat_matrix = numpy.empty((len(dataset),6))
    for i in range(len(dataset)):
        sentences = sent_tokenize(dataset[i]['reviewText'])
        nsent = len(sentences)
        if nsent:
            sentences_polarities = numpy.empty((nsent,3))
            for j in range(nsent):
                polarity = sia.polarity_scores(sentences[j])
                sentences_polarities[j,0] = polarity['pos']
                sentences_polarities[j,1] = polarity['neu']
                sentences_polarities[j,2] = polarity['neg']
            feat_matrix[i, 0:3] = numpy.mean(sentences_polarities, axis = 0) #axis 0 is row, axis 1 is columns
            feat_matrix[i,3:6] = numpy.max(sentences_polarities, axis = 0)
        else:
            feat_matrix[i, 0:6] = 0.0
            
    return feat_matrix
            
sia_tr = sia_features(baby_train)

### **Homework** : add lenght function

In [38]:
baby_train

[{'reviewerID': 'A1HK2FQW6KXQB2',
  'asin': '097293751X',
  'reviewerName': 'Amanda Johnsen "Amanda E. Johnsen"',
  'helpful': [0, 0],
  'reviewText': "Perfect for new parents. We were able to keep track of baby's feeding, sleep and diaper change schedule for the first two and a half months of her life. Made life easier when the doctor would ask questions about habits because we had it all right there!",
  'overall': 5.0,
  'summary': 'Awesine',
  'unixReviewTime': 1373932800,
  'reviewTime': '07 16, 2013',
  'hash': 5087991110168262753},
 {'reviewerID': 'A3OQEVD4C7G3L3',
  'asin': '097293751X',
  'reviewerName': 'CMB',
  'helpful': [0, 0],
  'reviewText': 'This is great for basics, but I wish the space to write things in was bigger. A lot times I need struggle trying to read what the caretaker wrote in because the spaces go together.',
  'overall': 4.0,
  'summary': 'Great for basics, but not detail',
  'unixReviewTime': 1399680000,
  'reviewTime': '05 10, 2014',
  'hash': -3952210150

In [39]:
def len_features(dataset):
    '''
    Add two features:
    (1) lenght of review (in thousands of characters) - truncate at 2500
    (2) percengate of exclamation marks (in %)    
    '''
    lenfeat_matrix =  numpy.empty(len(dataset),2)
    for i in range(len(dataset)):
        excla = 0
        for j in range(len(dataset[i]['reviewText'])):
            if 
            
        review_len = (len(dataset[i]['reviewText']))/1000 
        if (review_len >= 2500):
            review_len = 2500
len_tr = len_features(baby_train)

SyntaxError: invalid syntax (<ipython-input-39-76fd04afa186>, line 11)

In [40]:
# stack horizontally 
X_train_augmented = numpy.concatenate(( X_train_neg, sia_tr) ,axis = 1)
lreg_augmented = LinearRegression().fit(X_train_augmented,Y_train)
pred_train_augmented = lreg_augmented.predict(X_train_augmented)
mean_train_augmented = mean_absolute_error(pred_train_augmented, Y_train)
print("Now the mean absolute error on the training data is %f starts" % mean_train_augmented)

Now the mean absolute error on the training data is 0.754747 starts


In [41]:
# random forest
rf_augmented = RandomForestRegressor().fit(X_train_augmented, Y_train) #fetting
rfpred_train_augmented = rf_augmented.predict(X_train_augmented) # prediction
mae_train_rf_augmented = mean_absolute_error(rfpred_train_augmented, Y_train) #check error
print("For the RF, MAE is %f stars" % mae_train_rf_augmented)

For the RF, MAE is 0.290851 stars


In [42]:
X_valid_neg = dataset_to_matrix(baby_valid)
sia_valid = sia_features(baby_valid)
#leng_valid
X_valid_augmented = numpy.concatenate((X_valid_neg,sia_valid), axis = 1)
pred_valid_lraugmented = lreg_augmented.predict(X_valid_augmented)
pred_valid_rf_augmented = rf_augmented.predict(X_valid_augmented)

mae_valid_augmented = mean_absolute_error(pred_valid_lraugmented, Y_valid)
print("on the validation set, we get %f error for the linear regresion" % mae_valid_augmented)
mae_valid_rfaugemented = mean_absolute_error(pred_valid_rf_augmented, Y_valid)
print("and %f  for the random forest regresion" % mae_valid_rfaugemented)

on the validation set, we get 0.764692 error for the linear regresion
and 0.761418  for the random forest regresion


### **HOMEWORK**  ="Be lazy. Not just lazy but proactively, agressively lazy." Remove duplication.
create a single function that takes in data and spits out all success metrics across all of your algos.